In [95]:
# importing all the data 
import pandas as pd
import numpy as np
import os,datetime
from sklearn.linear_model import LinearRegression
from nepali_date_converter import english_to_nepali_converter

# intial_data = pd.read_excel('merged_data/ADBL.xlsx',skiprows=0)
# data = pd.DataFrame(columns=intial_data.columns)
# market_data = pd.read_excel('merged_data/NEPSE.xlsx',skiprows=0)
# # # # appending all the company data in single excel sheet
# for file in os.listdir('merged_data'):
#     company_data = pd.read_excel(f'merged_data/{file}',skiprows=0)
#     #mergin the stock price with nepse index
#     merged_data  = market_data.merge(company_data,on='Date',how='outer')[['Symbol_y','Date','Close_y','Close_x','Percent Change_x','Percent Change_y']].dropna(how='any').sort_values('Date')
#     merged_data.rename(columns={'Close_y' : 'company' ,'Close_x' : 'nepse','Symbol_y':'Script','Percent Change_x':'Rm','Percent Change_y':'Rj'},inplace=True)
#     merged_data['rm'] = merged_data.nepse.pct_change()*100
#     merged_data['rj'] = merged_data.company.pct_change()*100
#     merged_data['no_of_trading_days'] = range(len(merged_data))
#     # getting the event date from the csv
#     event_data = pd.read_excel('event_dates.xlsx')[['Date','Quote','EPS','Remarks']].dropna(how='any',axis = 0,subset=['Date','Quote','EPS'])
#     data = pd.concat([merged_data,data])
# data= data[['Script','Date','nepse','company','rm','rj','no_of_trading_days']] #this is the common database for all company
ar_data = pd.DataFrame(columns=['Company',"Event_date",'trading_date',10,9,8,7,6,5,4,3,2,1,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10])
# data.to_excel('data_shortcut.xlsx',index=False)
data = pd.read_excel('data_shortcut.xlsx')


In [30]:
# for comp in event_data.Quote:
#     comp_event_date = event_data[event_data['Quote']==comp]
#     event_data.loc[comp_event_date,'EPS_change'] = comp_event_date['EPS'] - comp_event_date['EPS'].shift(-1)

In [91]:
# modification of event_data  dataframe
# event_data['trading_day'] = None
# event_data.dropna(subset=['Date'],how = 'any',axis = 0,inplace=True)
# event_data = event_data[event_data['Quote'] != 'HBL']
# event_data = event_data[event_data['Quote'] != 'NIB']
# # locating the next trading day after news and appending it to the datframe

# for row,values in event_data.iterrows():
#     day =values['Date']
#     company_name = values['Quote']
#     while len(data[(data['Date'] == day) & (data['Script'] == company_name)]) != 1:
#         print(day,company_name)
#         day += datetime.timedelta(days= 1)
#     event_data.at[row,'trading_day'] = day
#     print(day,company_name,'found')

# event_data.to_excel('event_shortcut.xlsx',index=False)

event_data = pd.read_excel('event_shortcut.xlsx')
event_data.drop_duplicates(subset=['trading_day','Quote'],inplace=True)

In [92]:
event_data

,Date,Quote,EPS,Remarks,trading_day
0,2023-08-05,ADBL,20.61,good,2023-08-06
1,2023-04-21,ADBL,7.23,good,2023-04-23
2,2023-01-21,ADBL,-6.88,bad,2023-01-23
3,2022-10-24,ADBL,-3.16,bad,2022-10-31
4,2022-08-04,ADBL,17.92,good,2022-08-04
...,...,...,...,...,...
856,2020-11-13,UPPER,-0.64,good,2020-11-18
857,2020-08-16,UPPER,-0.65,bad,2020-08-16
858,2020-02-12,UPPER,-0.22,bad,2020-02-12
859,2019-11-16,UPPER,-0.12,NaN,2019-11-17


In [93]:
event_data['trading_day'] = pd.to_datetime(event_data['trading_day'])
data_with_event_date = data.merge(event_data,right_on = ['Quote','trading_day'],left_on=['Script','Date'],how='left').drop(['trading_day','Quote'],axis=1)
data_with_event_date.rename(columns={'Date_x':'Date','Date_y':'Event_date'},inplace=True)

In [59]:
# event_data

,Date,Quote,EPS,Remarks,trading_day
0,2023-08-05,ADBL,20.61,good,2023-08-06
1,2023-04-21,ADBL,7.23,good,2023-04-23
2,2023-01-21,ADBL,-6.88,bad,2023-01-23
3,2022-10-24,ADBL,-3.16,bad,2022-10-31
4,2022-08-04,ADBL,17.92,good,2022-08-04
...,...,...,...,...,...
856,2020-11-13,UPPER,-0.64,good,2020-11-18
857,2020-08-16,UPPER,-0.65,bad,2020-08-16
858,2020-02-12,UPPER,-0.22,bad,2020-02-12
859,2019-11-16,UPPER,-0.12,NaN,2019-11-17


In [96]:
# #ar_data is already loaded above


data_event_only = data_with_event_date[(data_with_event_date['Event_date'].notnull()) & (data_with_event_date['no_of_trading_days']>=250)]
data_with_event_date['er'] = np.nan
for ind,rows in data_event_only.iterrows():
    regression_period = data_with_event_date[ind-250:ind-10].dropna(subset=['rm','rj'],axis=0)
    event_window_period= data_with_event_date[ind-10:ind+11]
    if event_window_period.Script.nunique()==1:
        model = LinearRegression()
        x = np.array(regression_period['rm']).reshape(-1,1)
        y = np.array(regression_period['rj']).reshape(-1,1)
        model.fit(x,y)
        to_predict = np.array(event_window_period['rm']).reshape(-1,1)
        # event_window_period['er'] = model.predict(to_predict)
        data_with_event_date.loc[ind-10:ind+10,'er'] = model.predict(to_predict)
        predictions = model.predict(to_predict).flatten()
        other_detail = np.array([rows['Script'],rows['Event_date'],rows['Date']])
        final_detail = np.concatenate([other_detail,predictions],axis =0)
        ar_data.loc[len(ar_data)] =  final_detail
    else:
        print(ind)
data_with_event_date['Ar'] = data_with_event_date['rj']-data_with_event_date['er']
ar_data.dropna(subset=['Company'],axis = 0,inplace=True)
data_with_event_date.to_excel('data_with_event_date.xlsx', index = False)
data_with_event_date = pd.read_excel('data_with_event_date.xlsx')


In [97]:
# # getting the final dataset for analysis
ar_data['quarter'] = ar_data.Event_date.dt.quarter
quarter_mapping = {4:'q1',1:'q2',2:'q3',3:'q4'}
ar_data['quarter'] = ar_data['quarter'].map(quarter_mapping)
ar_data['fiscal_year'] = None
for index,rows  in ar_data.iterrows():
    if rows['quarter'] not in  ['q1']:
        ar_data.at[index,'fiscal_year'] = f'0{rows.Event_date.year + 56-2000}/0{rows.Event_date.year + 57-2000}'
    else:
        ar_data.at[index,'fiscal_year'] = f'0{rows.Event_date.year + 57-2000}/0{rows.Event_date.year + 58-2000}'
nepali_dates = []
for date in ar_data.Event_date:
    splitted = str(date).split('-')
    nepali = english_to_nepali_converter(int(splitted[0]),int(splitted[1]),int(splitted[2][:2]))
    nepali_dates.append(nepali)
ar_data['Nepali_date'] = nepali_dates
ar_data.to_excel('ar_data.xlsx',index = False)
final_data = ar_data.merge(event_data, left_on = ['Company','Event_date'],right_on = ['Quote','Date'], how = 'left')
final_data.drop(columns=['Date','Quote','trading_day'],inplace=True)
bull1= (final_data['Event_date'] >= '2011-07-01') & (final_data['Event_date'] <= '2016-07-03')
bull2= (final_data['Event_date'] >= '2019-02-03') & (final_data['Event_date'] <= '2021-08-1')
bear1= (final_data['Event_date'] >= '2016-07-03') & (final_data['Event_date'] <= '2019-02-03')
bear2= final_data['Event_date'] >= '2021-08-1'
final_data.drop_duplicates(inplace=True)
final_data.loc[bull1, 'market_condition'] = 'bm1'
final_data.loc[bull2, 'market_condition'] = 'bm2'
final_data.loc[bear1, 'market_condition'] = 'dm1'
final_data.loc[bear2, 'market_condition'] = 'dm2'
final_data.drop(columns =['quarter','fiscal_year','Nepali_date'] ,inplace=True)
final_data.to_excel('final.xlsx',index=False)